In [2]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet #ImageNet's model
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam


# first attempt at an image classifier
from sklearn import datasets
import tensorflow as tf
import numpy as npa
import pandas as pd

#get paths for files
import richardson_path as my_paths
import richardson_file_handlers as file_handler
import richardson_image_handlers as image_handler
from Richardson_Logger import r_logger

from keras import backend as K
#print(K.tensorflow_backend._get_available_gpus())
import matplotlib.pyplot as plt

Using TensorFlow backend.
4.1.2


In [3]:
#dimensions of input image
WINDOW_X = 128
WINDOW_Y = 128

max_images = 1000

In [4]:
print("Loading dictionary for y_train...")
my_logger = r_logger.R_logger(my_paths.INFO_PATH + '\\' + "data.csv")
my_y_values = my_logger.load_dictionary()

print("Loading images for train...")
x_train, x_train_files = file_handler.load_images_for_keras(my_paths.TRAIN_PATH, "png", max_images, WINDOW_X, WINDOW_Y, num_channels=3)
x_train = x_train / 255

print("Loading images for test...")
x_test, x_test_files = file_handler.load_images_for_keras(my_paths.VALIDATION_PATH, "png",max_images, WINDOW_X, WINDOW_Y, num_channels=3)
x_test = x_test / 255

Loading dictionary for y_train...
Loading images for train...
Loading images for test...


In [5]:
print('x_train shape:', x_train.shape)
print('Number of images in x_train', x_train.shape[0])
print('Number of images in x_test', x_test.shape[0])

x_train shape: (375, 128, 128, 3)
Number of images in x_train 375
Number of images in x_test 83


In [6]:
print("Get y values...")
y_train = image_handler.get_y_value(x_train_files, my_y_values, my_paths.human_labels)
y_test = image_handler.get_y_value(x_test_files, my_y_values, my_paths.human_labels)

Get y values...


## Transfer Learning

`input_shape` is required to be squared and must have any of these sizes: [128, 160, 192, 224]
We need to reshape the images.
https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e



In [7]:
input_shape = (WINDOW_Y, WINDOW_X, 3)

In [8]:
#Import the MobileNet model with Imagenet's weights and without the output layer

base_model=MobileNet(weights='imagenet',include_top=False, input_shape= input_shape) 

In [9]:
x=base_model.output

In [11]:
#Include a few dense layers and an output layer with softmax and two outputs (human head or no human head)

x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x)
preds=Dense(2,activation='softmax')(x) 



In [12]:
model=Model(inputs=base_model.input,outputs=preds)

In [13]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 129, 129, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 64, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 64, 64, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 64, 64, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 64, 64, 32)        128 

In [14]:
len(model.layers)

91

In [15]:
#Make the first 85 layers non-trainable and the rest trainable
for layer in model.layers[:85]:
    layer.trainable=False
for layer in model.layers[85:]:
    layer.trainable=True

ImageDataGenerators are inbuilt in keras and help us to train our model. We just have to specify the path to our training data and it automatically sends the data for training, in batches. It makes the code much simpler.

In [ ]:
'''train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('path-to-the-main-data-folder',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)'''

In [16]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy' ])

In [17]:
model.fit(x=x_train, y=y_train, batch_size = 32, epochs = 3)



Epoch 1/3
375/375 [==============================] - 11s 29ms/step - loss: 1.9177 - accuracy: 0.5867
Epoch 2/3
375/375 [==============================] - 9s 24ms/step - loss: 0.2831 - accuracy: 0.8853
Epoch 3/3
375/375 [==============================] - 9s 24ms/step - loss: 0.1414 - accuracy: 0.9573


In [18]:
print(model.evaluate(x_test, y_test))
print(model.metrics_names)

83/83 [==============================] - 2s 27ms/step
[0.3279098760650819, 0.8674699068069458]
['loss', 'accuracy']


In [19]:
model.save('stream2_classifier.h5')